In [1]:
import torch, os
from torchvision import transforms
from torch.autograd import Variable
from torch.nn import functional as F
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
import numpy as np
import cv2, torch
import numpy as np
import pandas as pd
import io
import requests
from PIL import Image
from collections import OrderedDict
from skimage import img_as_float,io

In [2]:
default_path= '/home/DL-based-Tumor-Classification/Datasets/Newest_case/Oral+Carcinoma'
def_path = default_path + '/img_fold0'
final_conv = 'conv3'
num_epochs = 50
batch_size = 400
learning_rate = 0.0001
imsize = 98
conv1_kernel = 5

Init Neural Net

In [3]:
class Net(nn.Module):

    def __init__(self, num_of_classes):
        super(Net, self).__init__()
        # input image channel, output channels, kernel size square convolution
        # kernel
        # input size = 28//96 output size = 28//96
        self.conv1 = nn.Conv2d(1, 64, kernel_size=conv1_kernel, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        # input size = 14//48 output size = 6//48
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        # input size = 6//24, output size = 3//24
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(256)
        self.drop2D = nn.Dropout2d(p=0.25, inplace=False)
        self.vp = nn.MaxPool2d(kernel_size=2, padding=0, stride=2)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(256*12*12, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 256)
        self.fc4 = nn.Linear(256, num_of_classes)


    def forward(self, x):
        in_size = x.size(0)
        x = F.relu(self.bn1(self.vp(self.conv1(x))))
        x = F.relu(self.bn2(self.vp(self.conv2(x))))
        x = F.relu(self.bn3(self.vp(self.conv3(x))))
        x = self.drop2D(x)
        x = x.view(in_size, -1)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        x = self.fc4(x)
        return x


net = Net(num_of_classes=6)
net = net.double()
# net = nn.DataParallel(net)
net.cuda()

Net(
  (conv1): Conv2d(1, 64, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (drop2D): Dropout2d(p=0.25, inplace=False)
  (vp): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=36864, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=256, bias=True)
  (fc4): Linear(in_features=256, out_features=6, bias=True)
)

Load Pretrained Weight

In [ ]:
state_dict = torch.load(def_path + '/network_0505_th1.pth')
new_state_dict = OrderedDict()
for k, v in state_dict.items():
    name = k[7:] # remove `module.`
    new_state_dict[name] = v
# load params
net.load_state_dict(new_state_dict)
net.eval()

Load Dataset

In [ ]:
def_path = default_path + '/img_fold0'
train_csv_path = def_path + '/train/labels_train.csv'
train_root_path = def_path + '/train'
test_csv_path = def_path + '/test/labels_test.csv'
test_root_path = def_path + '/test'
test_label_path =  def_path + '/test/test_label_run_th1_test.csv'
predicted_label_path = def_path + '/test/predicted_label_run_th1_test.csv'
model_path = def_path + '/network_0505_th1.pth'


class TumorDatasetTrain(Dataset):

    def __init__(self, csv_file , root_dir, transform=None):
        self.labels_frame = np.array(pd.read_csv(csv_file, skiprows=1, sep=',', header=None)).astype(np.int)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.labels_frame)

    def __getitem__(self, idx):
        img_name = str(idx)+'.png'
        img_path = os.path.join(self.root_dir, img_name)
        """
            !!!Pay attention!!!
            The image size is set here
            """
        img = np.empty(shape=(1, 96, 96))
        img[0, :, :] = (img_as_float(io.imread(img_path)) - 0.5)/0.5
        label = np.array([self.labels_frame[idx,1]-1])
        train_sample = {'image': img, 'label': label}

        if self.transform:
            train_sample = self.transform(train_sample)
        return train_sample


class TumorDatasetTest(Dataset):

    def __init__(self, csv_file , root_dir, transform=None):
        self.labels_frame = np.array(pd.read_csv(csv_file, skiprows= 1, sep=',', header= None)).astype(np.int)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.labels_frame)

    def __getitem__(self, idx):
        img_name = str(idx) + '.png'
        img_path = os.path.join(self.root_dir, img_name)
        """
            !!!Pay attention!!!
            The image size is set here
            """
        img = np.empty(shape=(1, 96, 96))
        img[0, :, :] = (img_as_float(io.imread(img_path)) - 0.5)/0.5
        label = np.array([self.labels_frame[idx, 1]-1])
        test_sample = {'image': img, 'label': label}

        if self.transform:
            test_sample = self.transform(test_sample)
        return test_sample


class ToTensor(object):

    def __call__(self, sample):
        image, labels = sample['image'], sample['label']
        return {'image': torch.from_numpy(image), 'label': torch.LongTensor(labels)}


train_dataset = TumorDatasetTrain(csv_file=train_csv_path, root_dir=train_root_path,
                                  transform=transforms.Compose([ToTensor()]))
test_dataset = TumorDatasetTest(csv_file=test_csv_path, root_dir=test_root_path, transform=transforms.Compose([ToTensor()]))
dataloader_train = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, num_workers=1)
dataloader_test = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=1)


In [ ]:
# generate class activation mapping for the top1 prediction
def returnCAM(feature_conv, weight_softmax, class_idx):
    # generate the class activation maps upsample to 256x256
    size_upsample = (256, 256)
    
#     print(feature_conv.shape)
    
    bz, nc, h, w = feature_conv.shape
    output_cam = []
    for idx in class_idx:
        cam = weight_softmax[class_idx].dot(feature_conv.reshape((nc, h*w)))
        cam = cam.reshape(h, w)
        cam = cam - np.min(cam)
        cam_img = cam / np.max(cam)
        cam_img = np.uint8(255 * cam_img)
        output_cam.append(cv2.resize(cam_img, size_upsample))
    return output_cam

def get_cam(net, features_blobs, img_pil, classes, root_img, item):
    params = list(net.parameters())
    param_temp = np.array(params)
    
    weight_softmax = np.squeeze(params[-2].data.cpu().numpy())
   
    normalize =  transforms.Normalize([0.5], [0.5])
    preprocess = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        normalize
    ])

    outputs = net(img_pil)  
#     probs, idx = torch.max(outputs.data, 1)    
    h_x = F.softmax(outputs, dim=1).data.squeeze()
    probs, idx = h_x.sort(0, True)
    print('Iki idx sakdurung ',idx)
    probs = probs.cpu().numpy()
    idx = idx.cpu().numpy()
    print('Iki idx ',idx)

    # output: the prediction
#     line = '{:.3f} -> {}'.format(probs[0], classes[idx[0]])
#     print(line)

    CAMs = returnCAM(features_blobs[0], weight_softmax, [idx[0]])

    # render the CAM and output
#     print('output CAM.jpg for the top1 prediction: %s' % classes[idx[0]])
    img = cv2.imread(root_img)
    height, width, _ = img.shape
    CAM = cv2.resize(CAMs[0], (width, height))
    heatmap = cv2.applyColorMap(CAM, cv2.COLORMAP_JET)
    result = heatmap * 0.3 + img * 0.5
    cv2.imwrite(def_path + '/coba5' + '/'+str(idx[0])+'_Cam_'+str(item)+'.jpg', result)
    print("DONE!")

In [ ]:
features_blobs = []

def hook_feature(module, input, output):
    features_blobs.append(output.data.cpu().numpy())

net._modules.get(final_conv).register_forward_hook(hook_feature)

for ii, test_sample in enumerate(dataloader_test):
        test_imgs = Variable(test_sample['image']).cuda()
        print("SEE THIS::: " , test_imgs.shape)
        test_label = Variable(test_sample['label']).cuda()
        print(test_label.shape)
        root = test_root_path + '/' + str(ii) + '.png'
        for i in range(0,test_imgs.size(0)):
            features_blobs = []
            print("now in loop "+str(i)+"!!!!!")
            test_imgs_temp = test_imgs[i,:,:,:]
            test_imgs_in = test_imgs_temp.unsqueeze(0)
            test_label_temp = test_label[i,:]
            test_label_in = test_label_temp.unsqueeze(0)
            get_cam(net, features_blobs, test_imgs_in, test_label_in, root, i)

LOOPED down here!!!!

In [ ]:
for i in range(10):
    print('FOLD {}'.format(i))
    def_path = default_path + '/img_fold{}'.format(i)
    train_csv_path = def_path + '/train/labels_train.csv'
    train_root_path = def_path + '/train'
    test_csv_path = def_path + '/test/labels_test.csv'
    test_root_path = def_path + '/test'
    test_label_path =  def_path + '/test/test_label_run_th1_test.csv'
    predicted_label_path = def_path + '/test/predicted_label_run_th1_test.csv'
    model_path = def_path + '/network_0505_th1.pth'
    

    state_dict = torch.load(def_path + '/network_0505_th1.pth')
    new_state_dict = OrderedDict()
    for k, v in state_dict.items():
        name = k[7:] # remove `module.`
        new_state_dict[name] = v
    # load params
    net.load_state_dict(new_state_dict)
    net.eval()
    
    class TumorDatasetTrain(Dataset):

        def __init__(self, csv_file , root_dir, transform=None):
            self.labels_frame = np.array(pd.read_csv(csv_file, skiprows=1, sep=',', header=None)).astype(np.int)
            self.root_dir = root_dir
            self.transform = transform

        def __len__(self):
            return len(self.labels_frame)

        def __getitem__(self, idx):
            img_name = str(idx)+'.png'
            img_path = os.path.join(self.root_dir, img_name)
            """
                !!!Pay attention!!!
                The image size is set here
                """
            img = np.empty(shape=(1, imsize, imsize))
            img[0, :, :] = (img_as_float(io.imread(img_path)) - 0.5)/0.5
            label = np.array([self.labels_frame[idx,1]-1])
            train_sample = {'image': img, 'label': label}

            if self.transform:
                train_sample = self.transform(train_sample)
            return train_sample


    class TumorDatasetTest(Dataset):

        def __init__(self, csv_file , root_dir, transform=None):
            self.labels_frame = np.array(pd.read_csv(csv_file, skiprows= 1, sep=',', header= None)).astype(np.int)
            self.root_dir = root_dir
            self.transform = transform

        def __len__(self):
            return len(self.labels_frame)

        def __getitem__(self, idx):
            img_name = str(idx) + '.png'
            img_path = os.path.join(self.root_dir, img_name)
            """
                !!!Pay attention!!!
                The image size is set here
                """
            img = np.empty(shape=(1, imsize, imsize))
            img[0, :, :] = (img_as_float(io.imread(img_path)) - 0.5)/0.5
            label = np.array([self.labels_frame[idx, 1]-1])
            test_sample = {'image': img, 'label': label}

            if self.transform:
                test_sample = self.transform(test_sample)
            return test_sample


    class ToTensor(object):

        def __call__(self, sample):
            image, labels = sample['image'], sample['label']
            return {'image': torch.from_numpy(image), 'label': torch.LongTensor(labels)}


    train_dataset = TumorDatasetTrain(csv_file=train_csv_path, root_dir=train_root_path,
                                      transform=transforms.Compose([ToTensor()]))
    test_dataset = TumorDatasetTest(csv_file=test_csv_path, root_dir=test_root_path, transform=transforms.Compose([ToTensor()]))
    dataloader_train = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, num_workers=1)
    dataloader_test = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=1)

    # generate class activation mapping for the top1 prediction
    def returnCAM(feature_conv, weight_softmax, class_idx):
        # generate the class activation maps upsample to 256x256
        size_upsample = (256, 256)

    #     print(feature_conv.shape)

        bz, nc, h, w = feature_conv.shape
        output_cam = []
        for idx in class_idx:
            cam = weight_softmax[class_idx].dot(feature_conv.reshape((nc, h*w)))
            cam = cam.reshape(h, w)
            cam = cam - np.min(cam)
            cam_img = cam / np.max(cam)
            cam_img = np.uint8(255 * cam_img)
            output_cam.append(cv2.resize(cam_img, size_upsample))
        return output_cam

    def get_cam(net, features_blobs, img_pil, classes, root_img, item):
        params = list(net.parameters())
        param_temp = np.array(params)

        weight_softmax = np.squeeze(params[-2].data.cpu().numpy())

        normalize =  transforms.Normalize([0.5], [0.5])
        preprocess = transforms.Compose([
            transforms.Resize((224,224)),
            transforms.ToTensor(),
            normalize
        ])

        outputs = net(img_pil)  
    #     probs, idx = torch.max(outputs.data, 1)    
        h_x = F.softmax(outputs, dim=1).data.squeeze()
        probs, idx = h_x.sort(0, True)
#         print('Iki idx sakdurung ',idx)
        probs = probs.cpu().numpy()
        idx = idx.cpu().numpy()
#         print('iki item: ',item)
#         print('Iki idx: ',idx[0])

        # output: the prediction
    #     line = '{:.3f} -> {}'.format(probs[0], classes[idx[0]])
    #     print(line)

        CAMs = returnCAM(features_blobs[0], weight_softmax, [idx[0]])

        # render the CAM and output
    #     print('output CAM.jpg for the top1 prediction: %s' % classes[idx[0]])
        img = cv2.imread(root_img)
        height, width, _ = img.shape
        CAM = cv2.resize(CAMs[0], (width, height))
        heatmap = cv2.applyColorMap(CAM, cv2.COLORMAP_JET)
        result = heatmap * 0.3 + img * 0.5
        if not os.path.exists(def_path + '/CAM2'):
            os.makedirs(def_path + '/CAM2')
        cv2.imwrite(def_path + '/CAM2' + '/'+str(idx[0])+'_Cam_'+str(item)+'.jpg', result)
        print("DONE!")

    features_blobs = []

    def hook_feature(module, input, output):
        features_blobs.append(output.data.cpu().numpy())

    net._modules.get(final_conv).register_forward_hook(hook_feature)

    for ii, test_sample in enumerate(dataloader_test):
            test_imgs = Variable(test_sample['image']).cuda()
#             print("SEE THIS::: " , test_imgs.shape)
            test_label = Variable(test_sample['label']).cuda()
#             print(test_label.shape)
            root = test_root_path + '/' + str(ii) + '.png'
            for i in range(0,test_imgs.size(0)):
                features_blobs = []
                print("now in loop "+str(i)+"!!!!!")
                test_imgs_temp = test_imgs[i,:,:,:]
                test_imgs_in = test_imgs_temp.unsqueeze(0)
                test_label_temp = test_label[i,:]
                test_label_in = test_label_temp.unsqueeze(0)
                get_cam(net, features_blobs, test_imgs_in, test_label_in, root, i)

In [ ]:
print(test_label.size())
imgs = test_label[0,:]
print(imgs.shape)
print(imgs.unsqueeze(0).size())
unsq = imgs.unsqueeze(0)
print(unsq)

In [ ]:
print(test_imgs.size(0)-1)